# Using RedisAI

Load the model into Redis and invoke it.

## Load The Model

Convert the model to ONNX and put it in Redis.

### Load The Pickle

In [1]:
import pickle

In [2]:
with open('model/woodchuckin.pickle', 'rb') as file:
  model = pickle.load(file)
model

RandomForestClassifier(max_depth=5, random_state=0)

### Convert to ONNX

In [3]:
import skl2onnx
from skl2onnx.common.data_types import FloatTensorType
skl2onnx.get_latest_tested_opset_version()

12

In [4]:
initial_type = [('float_input', FloatTensorType([None, 6]))]
onnx_model = skl2onnx.convert_sklearn(model, initial_types=initial_type, target_opset=12)

In [5]:
with open("model/woodchuckin.onnx", "wb") as f:
  f.write(onnx_model.SerializeToString())

### Put It in Redis

In [6]:
from redisai import Client
conn = Client(host='localhost', port=6379)

In [7]:
conn.modelset('woodchuckin', 'onnx', 'cpu', onnx_model.SerializeToString())

'OK'

## Use the Model

Let's use the model to make some predictions.

In [8]:
import numpy as np

### Load the Input Tensor

In [9]:
tensor_in = np.array([[44.6243, -89.9941, 19.2, 19.7, 34.3, 34.5]], dtype=np.float32)
conn.tensorset('woodchuckin:in', tensor_in, dtype='float')

'OK'

### Run the Model

In [10]:
conn.modelrun('woodchuckin', ['woodchuckin:in'], ['woodchuckin:out', 'woodchuckin:scores'])

'OK'

### Get the Output Tensor

In [11]:
conn.tensorget('woodchuckin:out')

array([0])